In [ ]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable
import re

Scoring based on traveller volume

In [ ]:
# Replace with the actual URI, username and password
AURA_CONNECTION_URI = "neo4j+s://85b099f8.databases.neo4j.io:7687"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = "password"

# Driver instantiation
driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)
#85b099f8.databases.neo4j.io:7687

In [ ]:
#calculate the total visitor volume of this dataset
query = (
        "MATCH (c1)-[r:travel_to]->(c2 {code :'SG'})"
        "RETURN max(r.visitors), min(r.visitors)"
        )
with driver.session() as session:
    results = session.run(query)
    for result in results:
        print(query)
                    
        maxvisitors=result['max(r.visitors)']
        minvisitors=result['min(r.visitors)']

MATCH (c1)-[r:travel_to]->(c2 {code :'SG'})RETURN max(r.visitors), min(r.visitors)


In [ ]:
  print(maxvisitors)
  print(minvisitors)

3627030
5182


In [ ]:
def onehop(country):
    try:
        query = (
                "MATCH (c1 {code :'" 
                f"{country}"
                "'})-[r:travel_to]-> (c2 {code :'SG'})"
                "RETURN c1, c2, r"
                )
        with driver.session() as session:
                results = session.run(query)
                for result in results:
                    print(query)

                    onehop_results=(result['r']['visitors']-minvisitors)/(maxvisitors-minvisitors)
                    return onehop_results

        
    except:
        pass

In [ ]:
  def twohop(country):
      try:

          query = (
              "MATCH (c1 {code :'" 
              f"{country}"
              "'})-[r1:travel_to]->(c2)-[r2:travel_to]-> (c3 {code :'SG'})"
              "RETURN c1, c2, r1, r2, c3"
                  )
          with driver.session() as session:
                  results = session.run(query)
                  for result in results:
                      first_hop=result['r1']['visitors']
                      second_hop=result['r2']['visitors']
                      twohop_results=((first_hop+second_hop)-(minvisitors*2))/((maxvisitors-minvisitors)*2)
                      return twohop_results
          
      except:
          pass

In [ ]:
country_list=[['US'],['US','FR'],['VN','SG']]
Singapore=['SG']
for country in country_list:
    match_code=[w for w in Singapore if w in country]
    if len(match_code)>0:
        print('contains SG')
    else:
        print('please carry on')

please carry on
please carry on
contains SG


In [ ]:
country_list=[['US'],['US','FR'],['SG','VN'],['BN']]
Singapore=['SG']
for country in country_list:
    match_code=[w for w in Singapore if w in country]
    if len(match_code)>0:
        print('1')
    else:
        if len(country)==1:
            for code in country:
                onehop_dist=onehop(code)
                print(onehop_dist)
                twohop_dist=twohop(code)
                print(twohop_dist)
                consolidate_visitor=(onehop_dist+twohop_dist)/2
                print(consolidate_visitor)
        else:
            consolidate_visitors=[]
            for code in country:
                onehop_dist=onehop(code)
                print(onehop_dist)
                twohop_dist=twohop(code)
                print(twohop_dist)
                consolidate_visitors.append(onehop_dist+twohop_dist)
                print(consolidate_visitors)
                consolidate_visitor=max(consolidate_visitors)
                print(consolidate_visitor)

MATCH (c1 {code :'US'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.19991948861465197
0.009196134128213001
0.10455781137143248
MATCH (c1 {code :'US'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.19991948861465197
0.009196134128213001
[0.20911562274286496]
0.20911562274286496
MATCH (c1 {code :'FR'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.0573188052066238
0.008782809217835757
[0.20911562274286496, 0.06610161442445955]
0.20911562274286496
1
MATCH (c1 {code :'BN'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.018615082687070246
0.43062685126487915
0.2246209669759747


In [ ]:
country_list=[['US'],['Pacific','FR'],['SG','VN'],['BN','VN']]
Singapore=['SG']
consolidate_list=[]
for country in country_list:
    match_code=[w for w in Singapore if w in country]
    if len(match_code)>0:
        print('1')
        consolidate_list.append(2)
    else:
        if len(country)==1:
            for code in country:
                onehop_dist=onehop(code)
                print(onehop_dist)
                twohop_dist=twohop(code)
                print(twohop_dist)
                consolidate_visitor=(onehop_dist+twohop_dist)/2
                print(consolidate_visitor)
                consolidate_list.append(consolidate_visitor)
                print(consolidate_list)
        else:
            consolidate_visitors=[]
            for code in country:
                onehop_dist=onehop(code)
                print(onehop_dist)
                twohop_dist=twohop(code)
                print(twohop_dist)
                if onehop_dist is None or twohop_dist is None:
                    consolidate_visitors.append(0)
                else:
                    consolidate_visitors.append((onehop_dist+twohop_dist)/2)
                    print(consolidate_visitors)
                    consolidate_visitor=max(consolidate_visitors)
                    print(consolidate_visitor)
            consolidate_list.append(consolidate_visitor) 
            print(consolidate_list)

MATCH (c1 {code :'US'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.19991948861465197
0.009196134128213001
0.10455781137143248
[0.10455781137143248]
None
None
MATCH (c1 {code :'FR'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.0573188052066238
0.008782809217835757
[0, 0.03305080721222978]
0.03305080721222978
[0.10455781137143248, 0.03305080721222978]
1
MATCH (c1 {code :'BN'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.018615082687070246
0.43062685126487915
[0.2246209669759747]
0.2246209669759747
MATCH (c1 {code :'VN'})-[r:travel_to]-> (c2 {code :'SG'})RETURN c1, c2, r
0.16200182890060544
0.009357515831696969
[0.2246209669759747, 0.0856796723661512]
0.2246209669759747
[0.10455781137143248, 0.03305080721222978, 2, 0.2246209669759747]


In [ ]:
consolidate_list

[0.10455781137143248, 0.03305080721222978, 2, 0.2246209669759747]

In [ ]:
onehop=[]
for country in country_list:

    for codes in country:
        query = (
                "MATCH (c1 {code :'" 
                f"{codes}"
                "'})-[r:travel_to]-> (c2 {code :'SG'})"
                "RETURN c1, c2, r"
                )
        with driver.session() as session:
            results = session.run(query)
            for result in results:
                onehop.append(result['r']['visitors'])
                print("The number of visitors from {0} to {2} is {1} for one hop.".format(result['c1']['code'], 
                                                                           result['r']['visitors'], 
                                                                           result['c2']['code']))

The number of visitors from US to SG is 729260 for one hop.
The number of visitors from FR to SG is 212782 for one hop.
The number of visitors from VN to SG is 591928 for one hop.
The number of visitors from BN to SG is 72603 for one hop.
The number of visitors from VN to SG is 591928 for one hop.


In [ ]:
country_list=[['US'],['US','FR'],['SG','US','VN']]
twohop=[]
for country in country_list:
    for codes in country:
        try:
        
            query = (
                "MATCH (c1 {code :'" 
                f"{codes}"
                "'})-[r1:travel_to]->(c2)-[r2:travel_to]-> (c3 {code :'SG'})"
                "RETURN c1, c2, r1, r2, c3"
                        )
            with driver.session() as session:
                results = session.run(query)
                for result in results:
                    first_hop=result['r1']['visitors']
                    second_hop=result['r2']['visitors']
                    each_twohop=first_hop+second_hop
                    twohop.append(each_twohop)
                    print("The number of visitors from {0} to {2} is {1}. The number of visitors from {2} to {4} is {3}".format(result['c1']['code'], 
                                                                   result['r1']['visitors'], 
                                                                   result['c2']['code'],
                                                                    result['r2']['visitors'],
                                                                    result['c3']['code']))
            

            

        except:
            pass


The number of visitors from US to BN is 4375. The number of visitors from BN to SG is 72603
The number of visitors from US to ID is 457832. The number of visitors from ID to SG is 3110416
The number of visitors from US to MY is 269928. The number of visitors from MY to SG is 1220664
The number of visitors from US to MM is 66757. The number of visitors from MM to SG is 155985
The number of visitors from US to PH is 1064440. The number of visitors from PH to SG is 829304
The number of visitors from US to TH is 1165856. The number of visitors from TH to SG is 528486
The number of visitors from US to VN is 746171. The number of visitors from VN to SG is 591928
The number of visitors from US to BN is 4375. The number of visitors from BN to SG is 72603
The number of visitors from US to ID is 457832. The number of visitors from ID to SG is 3110416
The number of visitors from US to MY is 269928. The number of visitors from MY to SG is 1220664
The number of visitors from US to MM is 66757. The 

Now Deal with Map Distance
http://techslides.com/list-of-countries-and-capitals#google_vignette

In [ ]:
import math
# Use this method to reduce dependencies as much as possible
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d


if __name__ == '__main__':
    import doctest
    doctest.testmod()

In [ ]:
lyon = (45.7597, 4.8422) # (latitude, longitude)
paris = (48.8567, 2.3508)
test=1/distance(lyon, paris)
test

0.00254961085185845

In [ ]:
country_map_df = pd.read_csv("country-capitals.csv")
country_map_df.head()

,CountryName,CapitalName,CapitalLatitude,CapitalLongitude,CountryCode,ContinentName
0,Somaliland,Hargeisa,9.550000,44.050000,SO,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe


In [ ]:
#take out Singapore's coordinates first, since it will always be the destination
singapore_coordinates=list(zip(country_map_df[country_map_df['CountryCode']=='SG'].CapitalLatitude,country_map_df[country_map_df['CountryCode']=='SG'].CapitalLongitude))
singapore_coordinates

[(1.283333333, 103.85)]

In [ ]:
country_map_df['map_coordinates'] = list(zip(country_map_df.CapitalLatitude, country_map_df.CapitalLongitude))
coordinates=country_map_df['map_coordinates']
coordinates

0                 (9.55, 44.05)
1           (-54.283333, -36.5)
2           (-49.35, 70.216667)
3      (31.76666667, 35.233333)
4             (60.116667, 19.9)
                 ...           
239                  (0.0, 0.0)
240                  (0.0, 0.0)
241                  (0.0, 0.0)
242                (-7.3, 72.4)
243                  (0.0, 0.0)
Name: map_coordinates, Length: 244, dtype: object

In [ ]:
impt=[]
for coordinate in coordinates:
    for singapore_coordinate in singapore_coordinates:
        #need to ensure that Singapore is taken out of the equation, cos dividing number by zero will result in error
        match_coordinate=[w for w in singapore_coordinate if w in coordinate]
        if len(match_coordinate)>0:
            #print(1)
            pass
        else:
        #calculation for other coordinates against the coordinates of Singapore
            calculation=1/distance(coordinate, singapore_coordinate)
            #print(calculation)
            impt.append(calculation)

In [ ]:
#set the maximum distance and minimum distance based on the data available
max_impt=max(impt) #0.0031478582679599874
min_impt=min(impt) #5.068900510473246e-05
print(max_impt)
print(min_impt)

0.0031478582679599874
5.068900510473246e-05


In [ ]:
max_impt=0.0031478582679599874
min_impt=5.068900510473246e-05

In [ ]:
consolidate_distance=[]
for coordinate in coordinates:
    for singapore_coordinate in singapore_coordinates:
        #need to ensure that Singapore is taken out of the equation, cos dividing number by zero will result in error
        match_coordinate=[w for w in singapore_coordinate if w in coordinate]
        if len(match_coordinate)>0:
            print(1)
            consolidate_distance.append(2)
        else:
        #calculation for other coordinates against the coordinates of Singapore
            calculation=1/distance(coordinate, singapore_coordinate)
            normalise= (calculation-min_impt)/(max_impt-min_impt)
            #print(calculation)
            consolidate_distance.append(normalise)

1


In [ ]:
consolidate_distance

[0.032009379627084986,
 0.008267249646465318,
 0.033427412846566,
 0.024396338368153334,
 0.017446397204676604,
 0.02965594143963416,
 0.0021568795896605973,
 0.01795784112422141,
 0.009278686698628176,
 0.04627621366857831,
 0.01792676194468553,
 0.013306525982168029,
 0.017433423431949077,
 0.013208436829860032,
 0.01560471185882307,
 0.002172876371075525,
 0.0021498473271544987,
 0.00395078761803382,
 0.027511566284216943,
 0.0012400744350261534,
 0.035626878923865055,
 0.01686968641942711,
 0.030119729965163163,
 0.002537037316795391,
 0.03466171681887887,
 0.09555402689916201,
 0.001989917342714699,
 0.019516145190614315,
 0.014201214444394702,
 0.0020487168241194345,
 0.012380288667759465,
 0.003693465156198146,
 0.08201114577532394,
 0.001462611538616168,
 0.017468803540918034,
 0.02010864889129235,
 0.0031720478896798627,
 0.0020971566328982735,
 0.23317125496966515,
 0.01896700627770336,
 0.011363619716804933,
 0.15178084738718414,
 0.02255221207202926,
 0.26496926587730785,
 

In [ ]:
country=country_map_df['CountryCode']

In [ ]:
df=pd.DataFrame({
    'country': country,
    'disance_impt':consolidate_distance
})

In [ ]:
df[df['disance_impt']>0.5]

,country,disance_impt
135,MY,1.0
193,SG,2.0
